In [28]:
import pandas as pd
import glob

# Get all csv files in a folder
files = glob.glob("./*.csv")

# Read and combine
df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

In [29]:
import json

# keep only user + assistant and remove tool calling
df = df[df["role"].isin(["user", "assistant"])].copy()

# parse + sort (oldest first within each conversation)
df["date"] = pd.to_datetime(df["date"], utc=True)
# tiebreaker: user first, then assistant
role_order = {"user": 0, "assistant": 1}
df["_role_order"] = df["role"].map(role_order).fillna(9).astype(int)
df = df.sort_values(["conversation_id", "date","_role_order"], ascending=True)

# build a per-conversation "full_conversation" JSONL-style string (easy to parse)
def build_full_conversation(group: pd.DataFrame) -> str:
    msgs = [{"role": r, "content": c} for r, c in zip(group["role"], group["content"])]
    # one JSON object per line (JSONL) is very LLM-friendly
    return " | ".join(json.dumps(m, ensure_ascii=False) for m in msgs)

# Collapse each conversation into one row, building the full message history and extracting the assistant_id from assistant messages
df_conversations = (
    df.groupby("conversation_id")
      .apply(lambda g: pd.Series({
          "full_conversation": build_full_conversation(g),
          "assistant_id": g.loc[g["role"] == "assistant", "assistant_id"]
                           .dropna()
                           .iloc[0] if (g["role"] == "assistant").any() else None
      }))
      .reset_index()
)

/var/folders/st/2vrhtd1x3_zfcsn7k38kxp6r0000gn/T/ipykernel_38134/673821526.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [ ]:
# Map assistant_id -> assistant_name
assistant_id_to_name = {
    "copilot-oM7GGEtrflUlnaVuFtAI": "Official | TCS | SHS Care",
    "copilot-9fKNYSbzY53lRRLRsfQE": "Official | TCS | FFH Care",
    "copilot-Jw7Uzrs5ZUgLEFcHqjHU": "Official | TCS | MOB TS",
    "copilot-NNXvLh3kBJDxko5daPRa": "Official | TCS | MNH Care",
    "copilot-fQYsLWe2L6Q0wlBEvzog": "Official | TCS | Expert Messaging",
    "copilot-RZMEQxdnWT6p8tnvBebo": "Official | TCS | FFH Retention",
    "copilot-JP1gM8ftYzUqqtCEzQ06": "Official | TCS | SHS Retention",
    "copilot-hIzYPbRzlWajDlg8HzoE": "Official | TCS | MOB Retention",
    "copilot-8a2QV82fAf9wLsFK8txD": "Official | TCS | OHE Care",
    "copilot-yim2KQXxMLigD0EHK88V": "Official | TBS | WLN Care",
    "copilot-KAvIIh4vSnoiE7ihcYHO": "Official | TBS | WLS Care",
    "copilot-lr5b1qwlkDegU1gh5AbJ": "Official | TCS | Koodo Care",
    "copilot-0dG1Si9OVA9ezycuyh4b": "Official | TBS | Loyalty and Retention",
    "copilot-04BGmKRURyADOoIvKW1D": "Official | TCS | SHS TS",
    "copilot-YPLMLf3KHD1ZbNw4TDBr": "Official | TBS | WLS Assure",
    "copilot-5kwsylhfPCzHELTDPbo0": "Official | TCS | Sales",
    "copilot-OSIgDg4KKVEGUTXLjr2W": "Official | TCS | MOB Care",
    "copilot-490597fe4c554160b59b": "Official | TCS | FFH TS",
}

df_conversations["assistant_name"] = (
    df_conversations["assistant_id"]
      .map(assistant_id_to_name)
      .fillna(df_conversations["assistant_id"])  # fallback: keep the id if unknown
)

df_conversations = df_conversations.drop(columns=["assistant_id"])


In [31]:
df_conversations.to_csv("looker-dashboard-conversations.csv", index=False)

In [23]:
df_conversations

,conversation_id,full_conversation,assistant_name
0,chat-3W6MmkcwE8oqA3HM5kRP,"{""role"": ""user"", ""content"": ""matching offer f...",Official | TCS | FFH Retention
1,chat-3fBlWxnpeblcukKK71Qm,"{""role"": ""user"", ""content"": ""Free Mode - I'll ...",Official | TCS | FFH Retention
2,chat-3kl3R4zBYngOEsyiHVvK,"{""role"": ""user"", ""content"": ""what is the zise ...",Official | TCS | FFH Retention
